# EDA for angle calculation

load libraries

In [1]:
import pandas as pd
import numpy as np
#import plotly.graph_objects as go
#import plotly.express as px
import json
import mediapipe as mp

## 0. Data preperation

Create template

In [2]:
# df_conn_names = pd.DataFrame(set(df_conn_idx['conn_name']))
# df_conn_names['name'] = 'name'
# df_conn_names['consider'] = True

# df_conn_names.to_csv('df_conn_names.csv', sep='\t', encoding='utf-8')

# df_conn_names

load angle template with indices (manually created)

In [3]:
df_angle_matrix = pd.read_csv('../static/angles.csv', sep=',')
df_angle_matrix

,connector_1_name,connector_2_name,vector_1_name,vector_2_name,join_name,vector_1_idx,vector_2_idx,join_idx
0,LEFT_FOOT_TOP,LEFT_LOWER_LEG,LEFT_FOOT_INDEX,LEFT_KNEE,LEFT_ANKLE,31,25,27
1,LEFT_LOWER_LEG,LEFT_UPPER_LEG,LEFT_ANKLE,LEFT_HIP,LEFT_KNEE,27,23,25
2,LEFT_UPPER_LEG,HIPS,LEFT_KNEE,RIGHT_HIP,LEFT_HIP,25,24,23
3,RIGHT_FOOT_TOP,RIGHT_LOWER_LEG,RIGHT_FOOT_INDEX,RIGHT_KNEE,RIGHT_ANKLE,32,26,28
4,RIGHT_LOWER_LEG,RIGHT_UPPER_LEG,RIGHT_ANKLE,RIGHT_HIP,RIGHT_KNEE,28,24,26
5,RIGHT_UPPER_LEG,HIPS,RIGHT_KNEE,LEFT_HIP,RIGHT_HIP,26,23,24
6,HIPS,LEFT_FLANK,RIGHT_HIP,LEFT_SHOULDER,LEFT_HIP,24,11,23
7,HIPS,RIGHT_FLANK,LEFT_HIP,RIGHT_SHOULDER,RIGHT_HIP,23,12,24
8,LEFT_FLANK,SHOULDERS,LEFT_HIP,RIGHT_SHOULDER,LEFT_SHOULDER,23,12,11
9,RIGHT_FLANK,SHOULDERS,RIGHT_HIP,LEFT_SHOULDER,RIGHT_SHOULDER,24,11,12


JSON files to dataframe

In [4]:
#https://stackoverflow.com/questions/69265059/is-it-possible-to-create-a-plotly-animated-3d-scatter-plot-of-mediapipes-body-p

mp_pose = mp.solutions.pose

#_PRESENCE_THRESHOLD = 0.5
_VISIBILITY_THRESHOLD = 0.5

f_connections = open('connections.json')
data_connections = json.load(f_connections)
f_connections.close()


def landmarks_to_df(landmark_list,connections=None):
    if not landmark_list:
        return
    plotted_landmarks = {}
    for idx, landmark in enumerate(landmark_list):
        ##if (
        ##    landmark['Visibility'] < _VISIBILITY_THRESHOLD
        ##):
        # or (
        #     landmark.HasField("presence") and landmark.presence < _PRESENCE_THRESHOLD
        # ):
        ##    continue
        plotted_landmarks[idx] = (-landmark['Z'], landmark['X'], -landmark['Y'])
    if connections:
        out_cn = []
        idx_pair_list = []
        num_landmarks = len(landmark_list)
        # Draws the connections if the start and end landmarks are both visible.
        for connection in connections:
            start_idx = connection[0]
            end_idx = connection[1]
            if not (0 <= start_idx < num_landmarks and 0 <= end_idx < num_landmarks):
                raise ValueError(
                    f"Landmark index is out of range. Invalid connection "
                    f"from landmark #{start_idx} to landmark #{end_idx}."
                )
            if start_idx in plotted_landmarks and end_idx in plotted_landmarks:
                landmark_pair = [
                    plotted_landmarks[start_idx],
                    plotted_landmarks[end_idx],
                ]
                out_cn.append(
                    dict(
                        xs=[landmark_pair[0][0], landmark_pair[1][0]],
                        ys=[landmark_pair[0][1], landmark_pair[1][1]],
                        zs=[landmark_pair[0][2], landmark_pair[1][2]],
                    )
                )
                idx_pair_list.append(
                    dict(
                        start_idx=start_idx,
                        end_idx=end_idx,
                    )
                )

        
        df_conn = pd.DataFrame(out_cn)
        df_conn_idx = pd.DataFrame(idx_pair_list)
 
        #df_conn_idx["lm"] = idx_pair_list.map(lambda s: mp_pose.PoseLandmark(s).name).values[0]
        df_conn_idx["start_name"] = df_conn_idx['start_idx'].map(lambda s: mp_pose.PoseLandmark(s).name).values
        df_conn_idx["end_name"] = df_conn_idx['end_idx'].map(lambda s: mp_pose.PoseLandmark(s).name).values
        df_conn_idx = df_conn_idx.merge(df_conn, left_index=True, right_index=True)
        
        #df_conn.to_csv('plot_df_conn.csv', sep='\t', encoding='utf-8')
        #df_conn_idx.to_csv('plot_df_conn_idx.csv', sep='\t', encoding='utf-8')


        # cn2 = {"xs": [], "ys": [], "zs": []}
        # for pair in out_cn:
        #     for k in pair.keys():
        #         cn2[k].append(pair[k][0])
        #         cn2[k].append(pair[k][1])
        #         cn2[k].append(None)

    df = pd.DataFrame(plotted_landmarks).T.rename(columns={0: "z", 1: "x", 2: "y"})
    df["lm"] = df.index.map(lambda s: mp_pose.PoseLandmark(s).name).values
    #df.to_csv('plot_df.csv', sep='\t', encoding='utf-8')
   
    return df



calculate angles

In [5]:
# https://stackoverflow.com/questions/2827393/angles-between-two-n-dimensional-vectors-in-python/13849249#13849249

def unit_vector(vector):
    return vector / np.linalg.norm(vector)

def angle_between(v1, v2):
    v1_u = unit_vector(v1)
    v2_u = unit_vector(v2)
    return np.arccos(np.clip(np.dot(v1_u, v2_u), -1.0, 1.0))

## 1. Calculate angles of correct exercise performance

load the landmarks from excersise gathered with gather.py

In [6]:
f_landmarks = open('example/half_boat_right.json')
data_landmark_list = json.load(f_landmarks)
f_landmarks.close()

landmark coordinates to dataframe

In [7]:
df = landmarks_to_df(data_landmark_list,  data_connections['list'])
df

,z,x,y,lm
0,-0.009656,0.301140,-0.290040,NOSE
1,-0.039898,0.292358,-0.262785,LEFT_EYE_INNER
2,-0.039700,0.291650,-0.263258,LEFT_EYE
3,-0.039561,0.290780,-0.263783,LEFT_EYE_OUTER
4,0.013054,0.286330,-0.262531,RIGHT_EYE_INNER
5,0.013106,0.281252,-0.262885,RIGHT_EYE
6,0.013172,0.275316,-0.263285,RIGHT_EYE_OUTER
7,-0.178762,0.259822,-0.278576,LEFT_EAR
8,0.063617,0.237249,-0.276216,RIGHT_EAR
9,-0.063825,0.295100,-0.321334,MOUTH_LEFT


insert respective 3D coordinates for vectors

In [8]:
df_angles = df_angle_matrix.copy(deep=True)

df_angles = df_angles.merge(df, how="left", left_on='vector_1_idx', right_on=df.index)
df_angles.drop(['lm'], axis=1, inplace=True)
df_angles.rename(columns={'z':'vector_1_z','x':'vector_1_x','y':'vector_1_y'}, inplace=True)

df_angles = df_angles.merge(df, how="left", left_on='vector_2_idx', right_on=df.index)
df_angles.drop(['lm'], axis=1, inplace=True)
df_angles.rename(columns={'z':'vector_2_z','x':'vector_2_x','y':'vector_2_y'}, inplace=True)

df_angles = df_angles.merge(df, how="left", left_on='join_idx', right_on=df.index)
df_angles.drop(['lm'], axis=1, inplace=True)
df_angles.rename(columns={'z':'join_z','x':'join_x','y':'join_y'}, inplace=True)

df_angles

,connector_1_name,connector_2_name,vector_1_name,vector_2_name,join_name,vector_1_idx,vector_2_idx,join_idx,vector_1_z,vector_1_x,vector_1_y,vector_2_z,vector_2_x,vector_2_y,join_z,join_x,join_y
0,LEFT_FOOT_TOP,LEFT_LOWER_LEG,LEFT_FOOT_INDEX,LEFT_KNEE,LEFT_ANKLE,31,25,27,0.064302,0.886691,-0.408518,-0.130750,0.550323,-0.488407,-0.024863,0.803548,-0.538659
1,LEFT_LOWER_LEG,LEFT_UPPER_LEG,LEFT_ANKLE,LEFT_HIP,LEFT_KNEE,27,23,25,-0.024863,0.803548,-0.538659,-0.159337,0.411463,-0.739581,-0.130750,0.550323,-0.488407
2,LEFT_UPPER_LEG,HIPS,LEFT_KNEE,RIGHT_HIP,LEFT_HIP,25,24,23,-0.130750,0.550323,-0.488407,0.159557,0.371142,-0.770487,-0.159337,0.411463,-0.739581
3,RIGHT_FOOT_TOP,RIGHT_LOWER_LEG,RIGHT_FOOT_INDEX,RIGHT_KNEE,RIGHT_ANKLE,32,26,28,0.203006,0.890485,-0.457003,0.218156,0.516808,-0.466565,0.190211,0.787789,-0.560609
4,RIGHT_LOWER_LEG,RIGHT_UPPER_LEG,RIGHT_ANKLE,RIGHT_HIP,RIGHT_KNEE,28,24,26,0.190211,0.787789,-0.560609,0.159557,0.371142,-0.770487,0.218156,0.516808,-0.466565
5,RIGHT_UPPER_LEG,HIPS,RIGHT_KNEE,LEFT_HIP,RIGHT_HIP,26,23,24,0.218156,0.516808,-0.466565,-0.159337,0.411463,-0.739581,0.159557,0.371142,-0.770487
6,HIPS,LEFT_FLANK,RIGHT_HIP,LEFT_SHOULDER,LEFT_HIP,24,11,23,0.159557,0.371142,-0.770487,-0.312151,0.278948,-0.429913,-0.159337,0.411463,-0.739581
7,HIPS,RIGHT_FLANK,LEFT_HIP,RIGHT_SHOULDER,RIGHT_HIP,23,12,24,-0.159337,0.411463,-0.739581,0.219024,0.202253,-0.437278,0.159557,0.371142,-0.770487
8,LEFT_FLANK,SHOULDERS,LEFT_HIP,RIGHT_SHOULDER,LEFT_SHOULDER,23,12,11,-0.159337,0.411463,-0.739581,0.219024,0.202253,-0.437278,-0.312151,0.278948,-0.429913
9,RIGHT_FLANK,SHOULDERS,RIGHT_HIP,LEFT_SHOULDER,RIGHT_SHOULDER,24,11,12,0.159557,0.371142,-0.770487,-0.312151,0.278948,-0.429913,0.219024,0.202253,-0.437278


calculate vectors of connectors 

In [9]:
df_angles['connector_1_x'] = df_angles['vector_1_x'] - df_angles['join_x']
df_angles['connector_1_y'] = df_angles['vector_1_y'] - df_angles['join_y']
df_angles['connector_1_z'] = df_angles['vector_1_z'] - df_angles['join_z']

df_angles['connector_2_x'] = df_angles['vector_2_x'] - df_angles['join_x']
df_angles['connector_2_y'] = df_angles['vector_2_y'] - df_angles['join_y']
df_angles['connector_2_z'] = df_angles['vector_2_z'] - df_angles['join_z']

df_angles

,connector_1_name,connector_2_name,vector_1_name,vector_2_name,join_name,vector_1_idx,vector_2_idx,join_idx,vector_1_z,vector_1_x,...,vector_2_y,join_z,join_x,join_y,connector_1_x,connector_1_y,connector_1_z,connector_2_x,connector_2_y,connector_2_z
0,LEFT_FOOT_TOP,LEFT_LOWER_LEG,LEFT_FOOT_INDEX,LEFT_KNEE,LEFT_ANKLE,31,25,27,0.064302,0.886691,...,-0.488407,-0.024863,0.803548,-0.538659,0.083142,0.130141,0.089166,-0.253225,0.050252,-0.105887
1,LEFT_LOWER_LEG,LEFT_UPPER_LEG,LEFT_ANKLE,LEFT_HIP,LEFT_KNEE,27,23,25,-0.024863,0.803548,...,-0.739581,-0.130750,0.550323,-0.488407,0.253225,-0.050252,0.105887,-0.138860,-0.251174,-0.028587
2,LEFT_UPPER_LEG,HIPS,LEFT_KNEE,RIGHT_HIP,LEFT_HIP,25,24,23,-0.130750,0.550323,...,-0.770487,-0.159337,0.411463,-0.739581,0.138860,0.251174,0.028587,-0.040321,-0.030906,0.318894
3,RIGHT_FOOT_TOP,RIGHT_LOWER_LEG,RIGHT_FOOT_INDEX,RIGHT_KNEE,RIGHT_ANKLE,32,26,28,0.203006,0.890485,...,-0.466565,0.190211,0.787789,-0.560609,0.102696,0.103606,0.012795,-0.270980,0.094044,0.027945
4,RIGHT_LOWER_LEG,RIGHT_UPPER_LEG,RIGHT_ANKLE,RIGHT_HIP,RIGHT_KNEE,28,24,26,0.190211,0.787789,...,-0.770487,0.218156,0.516808,-0.466565,0.270980,-0.094044,-0.027945,-0.145667,-0.303922,-0.058599
5,RIGHT_UPPER_LEG,HIPS,RIGHT_KNEE,LEFT_HIP,RIGHT_HIP,26,23,24,0.218156,0.516808,...,-0.739581,0.159557,0.371142,-0.770487,0.145667,0.303922,0.058599,0.040321,0.030906,-0.318894
6,HIPS,LEFT_FLANK,RIGHT_HIP,LEFT_SHOULDER,LEFT_HIP,24,11,23,0.159557,0.371142,...,-0.429913,-0.159337,0.411463,-0.739581,-0.040321,-0.030906,0.318894,-0.132515,0.309668,-0.152814
7,HIPS,RIGHT_FLANK,LEFT_HIP,RIGHT_SHOULDER,RIGHT_HIP,23,12,24,-0.159337,0.411463,...,-0.437278,0.159557,0.371142,-0.770487,0.040321,0.030906,-0.318894,-0.168889,0.333208,0.059467
8,LEFT_FLANK,SHOULDERS,LEFT_HIP,RIGHT_SHOULDER,LEFT_SHOULDER,23,12,11,-0.159337,0.411463,...,-0.437278,-0.312151,0.278948,-0.429913,0.132515,-0.309668,0.152814,-0.076695,-0.007365,0.531176
9,RIGHT_FLANK,SHOULDERS,RIGHT_HIP,LEFT_SHOULDER,RIGHT_SHOULDER,24,11,12,0.159557,0.371142,...,-0.429913,0.219024,0.202253,-0.437278,0.168889,-0.333208,-0.059467,0.076695,0.007365,-0.531176


3D coordinates of vectors as array

In [10]:
array = df_angles[['connector_1_z','connector_1_x','connector_1_y','connector_2_z','connector_2_x','connector_2_y']].to_numpy()
#array

calculate angles, copy that to static/excercises.json

In [11]:


angle_list = []

for angle_cal in array:
    getangle = angle_between((angle_cal[0],angle_cal[1],angle_cal[2]), (angle_cal[3],angle_cal[4],angle_cal[5]))
    getangle = getangle * (180/np.pi)
    angle_list.append(getangle)

angle_list

[118.77879783596462,
 108.52354914617311,
 92.6127346625851,
 114.83807054284385,
 95.38560982618957,
 91.77440698871477,
 116.32116362032973,
 97.27856370018833,
 68.33564555889038,
 78.03631030788665,
 93.26450087717774,
 127.72816881872254,
 150.51677485254126,
 101.14424856690425,
 143.5732435779299,
 129.02698167537883]

In [12]:
df_angles['angle'] = angle_list
df_angles

,connector_1_name,connector_2_name,vector_1_name,vector_2_name,join_name,vector_1_idx,vector_2_idx,join_idx,vector_1_z,vector_1_x,...,join_z,join_x,join_y,connector_1_x,connector_1_y,connector_1_z,connector_2_x,connector_2_y,connector_2_z,angle
0,LEFT_FOOT_TOP,LEFT_LOWER_LEG,LEFT_FOOT_INDEX,LEFT_KNEE,LEFT_ANKLE,31,25,27,0.064302,0.886691,...,-0.024863,0.803548,-0.538659,0.083142,0.130141,0.089166,-0.253225,0.050252,-0.105887,118.778798
1,LEFT_LOWER_LEG,LEFT_UPPER_LEG,LEFT_ANKLE,LEFT_HIP,LEFT_KNEE,27,23,25,-0.024863,0.803548,...,-0.130750,0.550323,-0.488407,0.253225,-0.050252,0.105887,-0.138860,-0.251174,-0.028587,108.523549
2,LEFT_UPPER_LEG,HIPS,LEFT_KNEE,RIGHT_HIP,LEFT_HIP,25,24,23,-0.130750,0.550323,...,-0.159337,0.411463,-0.739581,0.138860,0.251174,0.028587,-0.040321,-0.030906,0.318894,92.612735
3,RIGHT_FOOT_TOP,RIGHT_LOWER_LEG,RIGHT_FOOT_INDEX,RIGHT_KNEE,RIGHT_ANKLE,32,26,28,0.203006,0.890485,...,0.190211,0.787789,-0.560609,0.102696,0.103606,0.012795,-0.270980,0.094044,0.027945,114.838071
4,RIGHT_LOWER_LEG,RIGHT_UPPER_LEG,RIGHT_ANKLE,RIGHT_HIP,RIGHT_KNEE,28,24,26,0.190211,0.787789,...,0.218156,0.516808,-0.466565,0.270980,-0.094044,-0.027945,-0.145667,-0.303922,-0.058599,95.385610
5,RIGHT_UPPER_LEG,HIPS,RIGHT_KNEE,LEFT_HIP,RIGHT_HIP,26,23,24,0.218156,0.516808,...,0.159557,0.371142,-0.770487,0.145667,0.303922,0.058599,0.040321,0.030906,-0.318894,91.774407
6,HIPS,LEFT_FLANK,RIGHT_HIP,LEFT_SHOULDER,LEFT_HIP,24,11,23,0.159557,0.371142,...,-0.159337,0.411463,-0.739581,-0.040321,-0.030906,0.318894,-0.132515,0.309668,-0.152814,116.321164
7,HIPS,RIGHT_FLANK,LEFT_HIP,RIGHT_SHOULDER,RIGHT_HIP,23,12,24,-0.159337,0.411463,...,0.159557,0.371142,-0.770487,0.040321,0.030906,-0.318894,-0.168889,0.333208,0.059467,97.278564
8,LEFT_FLANK,SHOULDERS,LEFT_HIP,RIGHT_SHOULDER,LEFT_SHOULDER,23,12,11,-0.159337,0.411463,...,-0.312151,0.278948,-0.429913,0.132515,-0.309668,0.152814,-0.076695,-0.007365,0.531176,68.335646
9,RIGHT_FLANK,SHOULDERS,RIGHT_HIP,LEFT_SHOULDER,RIGHT_SHOULDER,24,11,12,0.159557,0.371142,...,0.219024,0.202253,-0.437278,0.168889,-0.333208,-0.059467,0.076695,0.007365,-0.531176,78.036310


change order of vectors to check if there is any difference

In [13]:
array_switched = df_angles[['connector_2_z','connector_2_x','connector_2_y','connector_1_z','connector_1_x','connector_1_y']].to_numpy()

In [14]:
angle_list_switched = []

for angle_cal in array_switched:
    getangle = angle_between((angle_cal[0],angle_cal[1],angle_cal[2]), (angle_cal[3],angle_cal[4],angle_cal[5]))
    angle_list_switched.append(getangle)

angle_list_switched

[2.073081103798298,
 1.8940932485506017,
 1.6163971491379896,
 2.004302437610137,
 1.6647929504896641,
 1.6017655710183671,
 2.030187294925801,
 1.6978312281793242,
 1.1926820114784777,
 1.361990550980612,
 1.6277726155358911,
 2.2292770934298645,
 2.6270133006598475,
 1.7653001569480353,
 2.5058258182026805,
 2.25194565414575]

## 2. Calculate angles of incorrect exercise performance

load the landmarks from excersise gathered with gather.py

In [15]:
f_landmarks_incorrect = open('example/half_boat_wrong.json')
data_landmark_list_incorrect = json.load(f_landmarks_incorrect)
f_landmarks_incorrect.close()

landmark coordinates to dataframe

In [16]:
df_incorrect = landmarks_to_df(data_landmark_list_incorrect,  data_connections['list'])
df_incorrect

,z,x,y,lm
0,1.032197,0.374523,-0.521763,NOSE
1,1.060417,0.340356,-0.526412,LEFT_EYE_INNER
2,1.060372,0.331501,-0.514454,LEFT_EYE
3,1.060664,0.322764,-0.502438,LEFT_EYE_OUTER
4,1.063918,0.360627,-0.564761,RIGHT_EYE_INNER
5,1.064181,0.366758,-0.580287,RIGHT_EYE
6,1.064306,0.372877,-0.595718,RIGHT_EYE_OUTER
7,0.951446,0.297563,-0.498862,LEFT_EAR
8,0.976251,0.370525,-0.630977,RIGHT_EAR
9,0.964623,0.368366,-0.495958,MOUTH_LEFT


insert respective 3D coordinates for vectors

In [17]:
df_angles_incorrect = df_angle_matrix.copy(deep=True)

df_angles_incorrect = df_angles_incorrect.merge(df_incorrect, how="left", left_on='vector_1_idx', right_on=df_incorrect.index)
df_angles_incorrect.drop(['lm'], axis=1, inplace=True)
df_angles_incorrect.rename(columns={'z':'vector_1_z','x':'vector_1_x','y':'vector_1_y'}, inplace=True)

df_angles_incorrect = df_angles_incorrect.merge(df_incorrect, how="left", left_on='vector_2_idx', right_on=df_incorrect.index)
df_angles_incorrect.drop(['lm'], axis=1, inplace=True)
df_angles_incorrect.rename(columns={'z':'vector_2_z','x':'vector_2_x','y':'vector_2_y'}, inplace=True)

df_angles_incorrect = df_angles_incorrect.merge(df_incorrect, how="left", left_on='join_idx', right_on=df_incorrect.index)
df_angles_incorrect.drop(['lm'], axis=1, inplace=True)
df_angles_incorrect.rename(columns={'z':'join_z','x':'join_x','y':'join_y'}, inplace=True)

df_angles

,connector_1_name,connector_2_name,vector_1_name,vector_2_name,join_name,vector_1_idx,vector_2_idx,join_idx,vector_1_z,vector_1_x,...,join_z,join_x,join_y,connector_1_x,connector_1_y,connector_1_z,connector_2_x,connector_2_y,connector_2_z,angle
0,LEFT_FOOT_TOP,LEFT_LOWER_LEG,LEFT_FOOT_INDEX,LEFT_KNEE,LEFT_ANKLE,31,25,27,0.064302,0.886691,...,-0.024863,0.803548,-0.538659,0.083142,0.130141,0.089166,-0.253225,0.050252,-0.105887,118.778798
1,LEFT_LOWER_LEG,LEFT_UPPER_LEG,LEFT_ANKLE,LEFT_HIP,LEFT_KNEE,27,23,25,-0.024863,0.803548,...,-0.130750,0.550323,-0.488407,0.253225,-0.050252,0.105887,-0.138860,-0.251174,-0.028587,108.523549
2,LEFT_UPPER_LEG,HIPS,LEFT_KNEE,RIGHT_HIP,LEFT_HIP,25,24,23,-0.130750,0.550323,...,-0.159337,0.411463,-0.739581,0.138860,0.251174,0.028587,-0.040321,-0.030906,0.318894,92.612735
3,RIGHT_FOOT_TOP,RIGHT_LOWER_LEG,RIGHT_FOOT_INDEX,RIGHT_KNEE,RIGHT_ANKLE,32,26,28,0.203006,0.890485,...,0.190211,0.787789,-0.560609,0.102696,0.103606,0.012795,-0.270980,0.094044,0.027945,114.838071
4,RIGHT_LOWER_LEG,RIGHT_UPPER_LEG,RIGHT_ANKLE,RIGHT_HIP,RIGHT_KNEE,28,24,26,0.190211,0.787789,...,0.218156,0.516808,-0.466565,0.270980,-0.094044,-0.027945,-0.145667,-0.303922,-0.058599,95.385610
5,RIGHT_UPPER_LEG,HIPS,RIGHT_KNEE,LEFT_HIP,RIGHT_HIP,26,23,24,0.218156,0.516808,...,0.159557,0.371142,-0.770487,0.145667,0.303922,0.058599,0.040321,0.030906,-0.318894,91.774407
6,HIPS,LEFT_FLANK,RIGHT_HIP,LEFT_SHOULDER,LEFT_HIP,24,11,23,0.159557,0.371142,...,-0.159337,0.411463,-0.739581,-0.040321,-0.030906,0.318894,-0.132515,0.309668,-0.152814,116.321164
7,HIPS,RIGHT_FLANK,LEFT_HIP,RIGHT_SHOULDER,RIGHT_HIP,23,12,24,-0.159337,0.411463,...,0.159557,0.371142,-0.770487,0.040321,0.030906,-0.318894,-0.168889,0.333208,0.059467,97.278564
8,LEFT_FLANK,SHOULDERS,LEFT_HIP,RIGHT_SHOULDER,LEFT_SHOULDER,23,12,11,-0.159337,0.411463,...,-0.312151,0.278948,-0.429913,0.132515,-0.309668,0.152814,-0.076695,-0.007365,0.531176,68.335646
9,RIGHT_FLANK,SHOULDERS,RIGHT_HIP,LEFT_SHOULDER,RIGHT_SHOULDER,24,11,12,0.159557,0.371142,...,0.219024,0.202253,-0.437278,0.168889,-0.333208,-0.059467,0.076695,0.007365,-0.531176,78.036310


calculate vectors of connectors 

In [18]:
df_angles_incorrect['connector_1_x'] = df_angles_incorrect['vector_1_x'] - df_angles_incorrect['join_x']
df_angles_incorrect['connector_1_y'] = df_angles_incorrect['vector_1_y'] - df_angles_incorrect['join_y']
df_angles_incorrect['connector_1_z'] = df_angles_incorrect['vector_1_z'] - df_angles_incorrect['join_z']

df_angles_incorrect['connector_2_x'] = df_angles_incorrect['vector_2_x'] - df_angles_incorrect['join_x']
df_angles_incorrect['connector_2_y'] = df_angles_incorrect['vector_2_y'] - df_angles_incorrect['join_y']
df_angles_incorrect['connector_2_z'] = df_angles_incorrect['vector_2_z'] - df_angles_incorrect['join_z']

df_angles_incorrect

,connector_1_name,connector_2_name,vector_1_name,vector_2_name,join_name,vector_1_idx,vector_2_idx,join_idx,vector_1_z,vector_1_x,...,vector_2_y,join_z,join_x,join_y,connector_1_x,connector_1_y,connector_1_z,connector_2_x,connector_2_y,connector_2_z
0,LEFT_FOOT_TOP,LEFT_LOWER_LEG,LEFT_FOOT_INDEX,LEFT_KNEE,LEFT_ANKLE,31,25,27,-1.085953,0.491118,...,-0.289740,-1.047819,0.469386,-0.324788,0.021733,0.009764,-0.038134,-0.003511,0.035048,0.531490
1,LEFT_LOWER_LEG,LEFT_UPPER_LEG,LEFT_ANKLE,LEFT_HIP,LEFT_KNEE,27,23,25,-1.047819,0.469386,...,-0.321776,-0.516329,0.465875,-0.289740,0.003511,-0.035048,-0.531490,-0.045621,-0.032036,0.519813
2,LEFT_UPPER_LEG,HIPS,LEFT_KNEE,RIGHT_HIP,LEFT_HIP,25,24,23,-0.516329,0.465875,...,-0.437266,0.003484,0.420254,-0.321776,0.045621,0.032036,-0.519813,0.070488,-0.115490,-0.007564
3,RIGHT_FOOT_TOP,RIGHT_LOWER_LEG,RIGHT_FOOT_INDEX,RIGHT_KNEE,RIGHT_ANKLE,32,26,28,-1.076778,0.519661,...,-0.359179,-1.033826,0.506733,-0.373335,0.012928,0.025984,-0.042952,0.009560,0.014156,0.554621
4,RIGHT_LOWER_LEG,RIGHT_UPPER_LEG,RIGHT_ANKLE,RIGHT_HIP,RIGHT_KNEE,28,24,26,-1.033826,0.506733,...,-0.437266,-0.479205,0.516292,-0.359179,-0.009560,-0.014156,-0.554621,-0.025551,-0.078087,0.475124
5,RIGHT_UPPER_LEG,HIPS,RIGHT_KNEE,LEFT_HIP,RIGHT_HIP,26,23,24,-0.479205,0.516292,...,-0.321776,-0.004080,0.490742,-0.437266,0.025551,0.078087,-0.475124,-0.070488,0.115490,0.007564
6,HIPS,LEFT_FLANK,RIGHT_HIP,LEFT_SHOULDER,LEFT_HIP,24,11,23,-0.004080,0.490742,...,-0.407841,0.003484,0.420254,-0.321776,0.070488,-0.115490,-0.007564,-0.124252,-0.086065,0.762748
7,HIPS,RIGHT_FLANK,LEFT_HIP,RIGHT_SHOULDER,RIGHT_HIP,23,12,24,0.003484,0.420254,...,-0.681469,-0.004080,0.490742,-0.437266,-0.070488,0.115490,0.007564,-0.081071,-0.244202,0.733967
8,LEFT_FLANK,SHOULDERS,LEFT_HIP,RIGHT_SHOULDER,LEFT_SHOULDER,23,12,11,0.003484,0.420254,...,-0.681469,0.766232,0.296002,-0.407841,0.124252,0.086065,-0.762748,0.113669,-0.273628,-0.036346
9,RIGHT_FLANK,SHOULDERS,RIGHT_HIP,LEFT_SHOULDER,RIGHT_SHOULDER,24,11,12,-0.004080,0.490742,...,-0.407841,0.729886,0.409671,-0.681469,0.081071,0.244202,-0.733967,-0.113669,0.273628,0.036346


3D coordinates of vectors as array

In [19]:
array_incorrect = df_angles_incorrect[['connector_1_z','connector_1_x','connector_1_y','connector_2_z','connector_2_x','connector_2_y']].to_numpy()
#array_incorrect

calculate angles

In [20]:
angle_list_incorrect = []

for angle_cal_incorrect in array_incorrect:
    getangle_incorrect = angle_between((angle_cal_incorrect[0],angle_cal_incorrect[1],angle_cal_incorrect[2]), (angle_cal_incorrect[3],angle_cal_incorrect[4],angle_cal_incorrect[5]))
    getangle_incorrect = getangle_incorrect * (180/np.pi)
    angle_list_incorrect.append(getangle_incorrect)

#angle_list_incorrect

In [21]:
df_angles_incorrect['angle'] = angle_list_incorrect
df_angles_incorrect

,connector_1_name,connector_2_name,vector_1_name,vector_2_name,join_name,vector_1_idx,vector_2_idx,join_idx,vector_1_z,vector_1_x,...,join_z,join_x,join_y,connector_1_x,connector_1_y,connector_1_z,connector_2_x,connector_2_y,connector_2_z,angle
0,LEFT_FOOT_TOP,LEFT_LOWER_LEG,LEFT_FOOT_INDEX,LEFT_KNEE,LEFT_ANKLE,31,25,27,-1.085953,0.491118,...,-1.047819,0.469386,-0.324788,0.021733,0.009764,-0.038134,-0.003511,0.035048,0.531490,146.629037
1,LEFT_LOWER_LEG,LEFT_UPPER_LEG,LEFT_ANKLE,LEFT_HIP,LEFT_KNEE,27,23,25,-1.047819,0.469386,...,-0.516329,0.465875,-0.289740,0.003511,-0.035048,-0.531490,-0.045621,-0.032036,0.519813,171.365220
2,LEFT_UPPER_LEG,HIPS,LEFT_KNEE,RIGHT_HIP,LEFT_HIP,25,24,23,-0.516329,0.465875,...,0.003484,0.420254,-0.321776,0.045621,0.032036,-0.519813,0.070488,-0.115490,-0.007564,87.210386
3,RIGHT_FOOT_TOP,RIGHT_LOWER_LEG,RIGHT_FOOT_INDEX,RIGHT_KNEE,RIGHT_ANKLE,32,26,28,-1.076778,0.519661,...,-1.033826,0.506733,-0.373335,0.012928,0.025984,-0.042952,0.009560,0.014156,0.554621,144.204163
4,RIGHT_LOWER_LEG,RIGHT_UPPER_LEG,RIGHT_ANKLE,RIGHT_HIP,RIGHT_KNEE,28,24,26,-1.033826,0.506733,...,-0.479205,0.516292,-0.359179,-0.009560,-0.014156,-0.554621,-0.025551,-0.078087,0.475124,168.482643
5,RIGHT_UPPER_LEG,HIPS,RIGHT_KNEE,LEFT_HIP,RIGHT_HIP,26,23,24,-0.479205,0.516292,...,-0.004080,0.490742,-0.437266,0.025551,0.078087,-0.475124,-0.070488,0.115490,0.007564,86.821248
6,HIPS,LEFT_FLANK,RIGHT_HIP,LEFT_SHOULDER,LEFT_HIP,24,11,23,-0.004080,0.490742,...,0.003484,0.420254,-0.321776,0.070488,-0.115490,-0.007564,-0.124252,-0.086065,0.762748,92.495707
7,HIPS,RIGHT_FLANK,LEFT_HIP,RIGHT_SHOULDER,RIGHT_HIP,23,12,24,0.003484,0.420254,...,-0.004080,0.490742,-0.437266,-0.070488,0.115490,0.007564,-0.081071,-0.244202,0.733967,99.247070
8,LEFT_FLANK,SHOULDERS,LEFT_HIP,RIGHT_SHOULDER,LEFT_SHOULDER,23,12,11,0.003484,0.420254,...,0.766232,0.296002,-0.407841,0.124252,0.086065,-0.762748,0.113669,-0.273628,-0.036346,85.479108
9,RIGHT_FLANK,SHOULDERS,RIGHT_HIP,LEFT_SHOULDER,RIGHT_SHOULDER,24,11,12,-0.004080,0.490742,...,0.729886,0.409671,-0.681469,0.081071,0.244202,-0.733967,-0.113669,0.273628,0.036346,82.344758


## 3. Calculate difference between angles and rate

new dataframe 

In [22]:
df_diff = df_angles[['angle']].copy(deep=True)
df_diff['angle2'] = df_angles_incorrect[['angle']]
df_diff

,angle,angle2
0,118.778798,146.629037
1,108.523549,171.365220
2,92.612735,87.210386
3,114.838071,144.204163
4,95.385610,168.482643
5,91.774407,86.821248
6,116.321164,92.495707
7,97.278564,99.247070
8,68.335646,85.479108
9,78.036310,82.344758


difference

In [23]:
df_diff['diff'] = df_diff['angle'] - df_diff['angle2']
df_diff

,angle,angle2,diff
0,118.778798,146.629037,-27.850239
1,108.523549,171.365220,-62.841670
2,92.612735,87.210386,5.402348
3,114.838071,144.204163,-29.366093
4,95.385610,168.482643,-73.097033
5,91.774407,86.821248,4.953159
6,116.321164,92.495707,23.825457
7,97.278564,99.247070,-1.968506
8,68.335646,85.479108,-17.143462
9,78.036310,82.344758,-4.308448


difference in %

In [24]:
df_diff['diff_per100'] = (df_diff['diff'] / 180) * 100
df_diff

,angle,angle2,diff,diff_per100
0,118.778798,146.629037,-27.850239,-15.472355
1,108.523549,171.365220,-62.841670,-34.912039
2,92.612735,87.210386,5.402348,3.001305
3,114.838071,144.204163,-29.366093,-16.314496
4,95.385610,168.482643,-73.097033,-40.609463
5,91.774407,86.821248,4.953159,2.751755
6,116.321164,92.495707,23.825457,13.236365
7,97.278564,99.247070,-1.968506,-1.093614
8,68.335646,85.479108,-17.143462,-9.524146
9,78.036310,82.344758,-4.308448,-2.393582


merge difference with template

In [25]:
df_angles_final = df_angle_matrix.copy(deep=True)
df_angles_final = df_angles_final.merge(df_diff, left_index=True, right_index=True)
df_angles_final

,connector_1_name,connector_2_name,vector_1_name,vector_2_name,join_name,vector_1_idx,vector_2_idx,join_idx,angle,angle2,diff,diff_per100
0,LEFT_FOOT_TOP,LEFT_LOWER_LEG,LEFT_FOOT_INDEX,LEFT_KNEE,LEFT_ANKLE,31,25,27,118.778798,146.629037,-27.850239,-15.472355
1,LEFT_LOWER_LEG,LEFT_UPPER_LEG,LEFT_ANKLE,LEFT_HIP,LEFT_KNEE,27,23,25,108.523549,171.365220,-62.841670,-34.912039
2,LEFT_UPPER_LEG,HIPS,LEFT_KNEE,RIGHT_HIP,LEFT_HIP,25,24,23,92.612735,87.210386,5.402348,3.001305
3,RIGHT_FOOT_TOP,RIGHT_LOWER_LEG,RIGHT_FOOT_INDEX,RIGHT_KNEE,RIGHT_ANKLE,32,26,28,114.838071,144.204163,-29.366093,-16.314496
4,RIGHT_LOWER_LEG,RIGHT_UPPER_LEG,RIGHT_ANKLE,RIGHT_HIP,RIGHT_KNEE,28,24,26,95.385610,168.482643,-73.097033,-40.609463
5,RIGHT_UPPER_LEG,HIPS,RIGHT_KNEE,LEFT_HIP,RIGHT_HIP,26,23,24,91.774407,86.821248,4.953159,2.751755
6,HIPS,LEFT_FLANK,RIGHT_HIP,LEFT_SHOULDER,LEFT_HIP,24,11,23,116.321164,92.495707,23.825457,13.236365
7,HIPS,RIGHT_FLANK,LEFT_HIP,RIGHT_SHOULDER,RIGHT_HIP,23,12,24,97.278564,99.247070,-1.968506,-1.093614
8,LEFT_FLANK,SHOULDERS,LEFT_HIP,RIGHT_SHOULDER,LEFT_SHOULDER,23,12,11,68.335646,85.479108,-17.143462,-9.524146
9,RIGHT_FLANK,SHOULDERS,RIGHT_HIP,LEFT_SHOULDER,RIGHT_SHOULDER,24,11,12,78.036310,82.344758,-4.308448,-2.393582


count how many hit threshold

In [26]:
list_of_angle_diff = abs(df_angles_final['diff_per100'])
list(list_of_angle_diff)

[15.472354814224087,
 34.91203912757457,
 3.0013046027801553,
 16.314495872764645,
 40.60946281161774,
 2.751754942805099,
 13.236364913246208,
 1.0936143862182965,
 9.524145547083052,
 2.3935820876339164,
 9.604542252534207,
 9.146340388054155,
 22.895857996245358,
 2.410623303730948,
 7.2323817547914855,
 14.623299377971009]

In [27]:
list_of_angle_diff_length = len(list(list_of_angle_diff))
list_of_angle_diff_length

16

how many angles are correct

In [30]:
angle_threshold = 5
list_of_angle_diff_threshold = sum(i < angle_threshold for i in list(list_of_angle_diff))
list_of_angle_diff_threshold

5

correctness in %

In [31]:
list_of_angle_diff_threshold / list_of_angle_diff_length * 100

31.25